# BM40A1401 GPU Computing

## Erik Kuitunen

### Exercise 1

Import needed libraries.

In [443]:
#!pip install pycuda
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np
import math
import timeit

#### Task 1
Implement vector differentiation similar to numpy diff function using shared memory.

Defining the kernel

In [444]:
kernel_code = ( """
  __global__ void vec_diff_sharedmem( const float* a, float* b, int data_size ) {
  
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    float b_elem = 0;
    
    __shared__ float a_sub1[ %(THREADS)s ];
    __shared__ float a_sub2[ %(THREADS)s ];

    if ( index > data_size-2 ) {
      return;
    }
      
    a_sub1[ threadIdx.y ] = a[ index ];
    a_sub2[ threadIdx.y ] = a[ index + 1 ];  
    
    b_elem = a_sub2[ threadIdx.y ] - a_sub1[ threadIdx.y ];
      
    b[ index ] = b_elem;
     
  } 
  
  // Also non-shared memory version for comparison
  
  __global__ void vec_diff( const float* a, float* b, int data_size ) {
  
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    float b_elem = 0;

    if ( index > data_size-2 ) {
      return;
    }
    
    b_elem = a[ index + 1 ] - a[ index ];
      
    b[ index ] = b_elem;
     
  }
""" )



Create data vectors and initalize thread and block sizes

In [445]:
THREADS = 16*16

data_dim = 2**27

a = np.float32( np.random.rand( 1, data_dim ) )

grid_dim = math.ceil( data_dim / THREADS ) 

block_dims = ( THREADS, 1, 1 )
grid_dims = ( grid_dim, 1, 1 )
size_shared = a[0].nbytes * THREADS

Allocate memory and copy data from host to device 

In [446]:

a_gpu = cuda.mem_alloc( a.nbytes )
cuda.memcpy_htod( a_gpu, a )

b = np.float32( np.empty( [ 1, data_dim-1] ) )
b_gpu = cuda.mem_alloc( b.nbytes )

b_noshared = np.float32( np.empty( [ 1, data_dim-1] ) )
b_gpu_noshared = cuda.mem_alloc( b_noshared.nbytes )
                        

Calling the CUDA kernel and copy the result back to host

In [447]:
# Specifying constant THREADS for shared memory kernel
kernel = kernel_code % {
        'THREADS': THREADS
        }

# Compile the shared memory kernel code
mod = SourceModule( kernel )
diff_gpu = mod.get_function( "vec_diff_sharedmem" )

# Measure gpu execution time
gpu_start = timeit.default_timer()   
 
diff_gpu( a_gpu, b_gpu, np.int32( data_dim ), 
        block = block_dims, grid = grid_dims )

gpu_time = ( timeit.default_timer() - gpu_start ) * 1000

cuda.memcpy_dtoh( b, b_gpu )

###### Doing the same as above for non-shared memory version
diff_gpu_noshared = mod.get_function( "vec_diff" )

# Measure gpu execution time
gpu_start = timeit.default_timer()   
 
diff_gpu_noshared( a_gpu, b_gpu_noshared, np.int32( data_dim ), 
                block = block_dims, grid = grid_dims )

gpu_time_noshared = ( timeit.default_timer() - gpu_start ) * 1000

cuda.memcpy_dtoh( b_noshared, b_gpu_noshared )

C:\Users\erikk\AppData\Local\Temp\ipykernel_18932\2989837996.py:7: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod = SourceModule( kernel )


Verifying the results

In [448]:

# Measure cpu time
cpu_start = timeit.default_timer()  
 
b_cpu = np.diff( a )

cpu_time = ( timeit.default_timer() - cpu_start ) * 1000

if ( b_cpu == b ).all() and (b_cpu == b_noshared ).all():
    print( "The vectors are the same. \nCPU: " + str( cpu_time )+ " ms\nGPU, no shared memory: " 
            + str( gpu_time_noshared ) + " ms\nGPU, shared memory: " + str( gpu_time ) + " ms")   
else:
    print( "The vector are not the same. Something is wrong." )

The vectors are the same. 
CPU: 93.51569999853382 ms
GPU, no shared memory: 0.12149999929533806 ms
GPU, shared memory: 0.1326000001427019 ms


Shared memory version is slower. Bug in code, or is this kind of problem just generally more inefficient with shared memory?

#### Task 2
Implement the three reduction models presented in the lectures. Time their performance against different vector sizes. Execution times can vary between executions, so run them for example 100 times and take the average time.

Add also CPU performance with numpy sum() - function.

In [ ]:
data_sizes = np.array( [ 2**10, 2**13, 2**16, 2**20, 2**22 ] )
exec_times_gpu = []
exec_times_cpu = []

threads_per_block = 1024







# Looping through data sizes, doig the calculations and saving results.
for size in data_sizes:     
    
    blocks = math.ceil( size / threads_per_block)





Defining the kernels

In [ ]:
kernel_code = ( """
  __global__ void vec_diff_sharedmem( const float* a, float* b, int data_size ) {
  
    
     
  }
""" )

#### Task 3:

Implement Sequential reduction with shared memory. Add the performance to the same plot used in the previous task.

